패키지 다운로드

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

# 1️⃣ LLM을 활용해서 답변을 생성하는 가장 기본적인 방법

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [21]:
from langchain_core.prompts import PromptTemplate

question = '인프런에는 어떤 강의가 있나요?'
prompt_with_template = f"아래 질문에 답변해주세요: \n\n {question}"

prompt_template = PromptTemplate(template=prompt_with_template, input_variables=["question"])

1. 하드 코딩

In [22]:
llm.invoke(prompt_template.invoke({"question": question}))

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등 다양한 기술과 주제에 대한 강의를 찾을 수 있습니다. 각 강의는 강사에 따라 내용과 수준이 다르므로, 자신의 필요에 맞는 강의를 선택할 수 있습니다. 인프런 웹사이트를 방문하면 최신 강의 목록과 상세 정보를 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 27, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxl038WC5ME78K9sOsnNpzsoRGIm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c928cbf2-c6c8-4ece-9a3a-23d3769c3693-0', usage_metadata={'input_tokens': 27, 'output_tokens': 130, 'total_tokens': 157, 'input_token_details': {'audio': 0

2. 체인 사용

In [23]:
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question": question})

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등 다양한 기술과 주제에 대한 강의를 찾을 수 있습니다. 각 강의는 강사에 따라 내용과 수준이 다르므로, 자신의 필요에 맞는 강의를 선택할 수 있습니다. 인프런 웹사이트를 방문하면 최신 강의 목록과 상세 정보를 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 27, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxl4SJsJSLmtz3iJiXQFbCGKgQEq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--92954364-c7d6-452a-919f-7ec154281da3-0', usage_metadata={'input_tokens': 27, 'output_tokens': 130, 'total_tokens': 157, 'input_token_details': {'audio': 0

# 2️⃣ WebBaseLoader를 사용한 RAG

In [24]:
%pip install -qU beatifulsoup4

ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4
Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 전처리

In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=")

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200  # 이전 청크와 겹치게 가져올 토큰 수. 유사도 검색을 할 때 원하는 문서를 가져올 수 있는 확률을 높히는 방식
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [27]:
document_list

[Document(metadata={'source': 'https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=', 'title': '인프런 - 라이프타임 커리어 플랫폼', 'description': '프로그래밍, 인공지능, 데이터, 마케팅, 디자인등 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 곳.', 'language': 'ko'}, page_content='인프런 - 라이프타임 커리어 플랫폼NN인프런 - 라이프타임 커리어 플랫폼')]

In [28]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name="inflearn-lectures",
    persist_directory="./inflearn-lectures"
)

## 2. 데이터 검색을 위한 Retriever 생성

In [29]:
retriever = database.as_retriever(search_kwargs={'k': 1})

## 3. 프롬프트 생성

1. 수동 생성

In [30]:
rag_prompt_template = PromptTemplate(
    template='''
    You are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
    ''',
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retriever, "question": question})

AIMessage(content='인프런에는 다양한 주제의 강의가 있습니다. 주로 프로그래밍, 데이터 과학, 인공지능, 웹 개발, 모바일 개발, 디자인 등 여러 분야의 강의를 제공하고 있습니다. 각 강의는 초급부터 고급까지 다양한 수준으로 구성되어 있으며, 실습 중심의 강의도 많아 학습에 도움이 됩니다. 특정 강의에 대한 정보가 필요하시다면, 인프런 웹사이트를 방문하여 검색해 보시는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 91, 'total_tokens': 194, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxlA7OCdpKWoBcKNmMTEI8xqEx64', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1543f824-c53e-4b47-9365-ddd41b4cc1b3-0', usage_metadata={'input_tokens': 91, 'output_tokens': 103, 'total_tokens': 194, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

2. LangChain Hub 제공 프롬프트 활용

In [31]:
from langchain import hub

rlm_rag_prompt = hub.pull("rlm/rag-prompt")

rlm_rag_chain = rlm_rag_prompt | llm
rlm_rag_chain.invoke({"context": retriever, "question": question})

/Users/kimjaehee/Desktop/Study_file/langchain/rag_langchain/.venv/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


AIMessage(content='인프런에는 다양한 강의가 있습니다. 주로 프로그래밍, 데이터 과학, 디자인, 마케팅 등 여러 분야의 강의를 제공합니다. 구체적인 강의 내용은 인프런 웹사이트에서 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 115, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxlEK3vbUPyj1BODMuj92NlVZDxN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--416ac032-e307-4c82-b988-43a7522a8197-0', usage_metadata={'input_tokens': 115, 'output_tokens': 50, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

3. `RunnablePassThrough` 활용

In [32]:
from langchain_core.runnables import RunnablePassthrough

rag_pass_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    rlm_rag_prompt |
    llm
)
rag_pass_chain.invoke(question)

AIMessage(content='인프런에는 프로그래밍, 인공지능, 데이터, 마케팅, 디자인 등 다양한 강의가 있습니다. 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 기회를 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 219, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxlHKkshs2VVxLVLAOpAKaT5BN6x', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--56dc2f1b-e4be-41b4-b3ec-ad7324fa89d4-0', usage_metadata={'input_tokens': 219, 'output_tokens': 45, 'total_tokens': 264, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})